## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
appTypeCount = application_df.APPLICATION_TYPE.value_counts()
appTypeCount

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(appTypeCount[appTypeCount < 600].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classificationCount = application_df.CLASSIFICATION.value_counts()
classificationCount

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df.CLASSIFICATION.value_counts().loc[lambda x : x>1] 


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classificationCount[classificationCount < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
trim_variables_df = application_df.drop(["AFFILIATION", "USE_CASE", "SPECIAL_CONSIDERATIONS"],axis=1)
dummies = pd.get_dummies(trim_variables_df)
dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = dummies["IS_SUCCESSFUL"]
X = dummies.drop(["IS_SUCCESSFUL"],axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputDimension = len(X_train_scaled[0])
layerOneUnits = 10
layerTwoUnits = 20

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=layerOneUnits, activation="relu", input_dim=inputDimension))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=layerTwoUnits, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                310       
                                                                 
 dense_1 (Dense)             (None, 20)                220       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 551
Trainable params: 551
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [14]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200) 

Epoch 1/200
804/804 [==============================] - 1s 917us/step - loss: 0.6489 - accuracy: 0.6215
Epoch 2/200
804/804 [==============================] - 1s 903us/step - loss: 0.6219 - accuracy: 0.6475
Epoch 3/200
804/804 [==============================] - 1s 912us/step - loss: 0.6181 - accuracy: 0.6476
Epoch 4/200
804/804 [==============================] - 1s 906us/step - loss: 0.6168 - accuracy: 0.6485
Epoch 5/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6162 - accuracy: 0.6491
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.6151 - accuracy: 0.6504
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.6147 - accuracy: 0.6492
Epoch 8/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6144 - accuracy: 0.6509
Epoch 9/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6138 - accuracy: 0.6509
Epoch 10/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6134 

804/804 [==============================] - 2s 3ms/step - loss: 0.6096 - accuracy: 0.6505
Epoch 82/200
804/804 [==============================] - 2s 3ms/step - loss: 0.6094 - accuracy: 0.6530
Epoch 83/200
804/804 [==============================] - 2s 3ms/step - loss: 0.6097 - accuracy: 0.6525
Epoch 84/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6094 - accuracy: 0.6527
Epoch 85/200
804/804 [==============================] - 2s 3ms/step - loss: 0.6095 - accuracy: 0.6511
Epoch 86/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6094 - accuracy: 0.6522
Epoch 87/200
804/804 [==============================] - 2s 3ms/step - loss: 0.6094 - accuracy: 0.6522
Epoch 88/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6096 - accuracy: 0.6525
Epoch 89/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6093 - accuracy: 0.6530
Epoch 90/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6095 - accuracy: 

804/804 [==============================] - 3s 3ms/step - loss: 0.6090 - accuracy: 0.6515
Epoch 161/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6087 - accuracy: 0.6537
Epoch 162/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6088 - accuracy: 0.6521
Epoch 163/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6087 - accuracy: 0.6523
Epoch 164/200
804/804 [==============================] - 2s 3ms/step - loss: 0.6088 - accuracy: 0.6529
Epoch 165/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6086 - accuracy: 0.6537
Epoch 166/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6087 - accuracy: 0.6532
Epoch 167/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6086 - accuracy: 0.6513
Epoch 168/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6089 - accuracy: 0.6532
Epoch 169/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6084 - a

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6127 - accuracy: 0.6566 - 779ms/epoch - 3ms/step
Loss: 0.612682044506073, Accuracy: 0.6565597653388977


In [16]:
# Export our model to HDF5 file
nn_model.save('Models/AlphabetSoupCharity.h5')
